# IRST Library Model Zoo - Complete Guide
## Explore, Compare, and Select the Perfect Model

Welcome to the comprehensive guide for exploring the IRST Library Model Zoo! This notebook will help you understand, compare, and select the best model for your infrared small target detection needs.

### 🏆 **Available Models:**
- **SERANKNet** - Search and ranking network with attention mechanisms
- **ACMNet** - Asymmetric contextual modulation network  
- **MSHNet** - Multi-scale hybrid network
- **U-Net variants** - Classic segmentation architectures
- **Transformer-based** - Vision transformer adaptations
- **Lightweight models** - Mobile and edge deployment ready

### 🎯 **What you'll learn:**
- 📊 **Model Comparison** - Performance, speed, and memory analysis
- 🔧 **Model Selection** - Choose the right model for your use case
- ⚡ **Performance Benchmarking** - Speed and accuracy trade-offs
- 📱 **Deployment Considerations** - Model size and inference requirements  
- 🎨 **Visualization** - Compare model outputs and attention maps
- 📈 **Ablation Studies** - Understand model component contributions

### 📋 **Use Cases Covered:**
- 🎯 **High Accuracy Research** - State-of-the-art performance
- ⚡ **Real-time Applications** - Low latency requirements
- 📱 **Edge Deployment** - Resource-constrained environments
- 🏭 **Production Systems** - Balanced performance and efficiency

## 1. Setup and Model Loading

In [ ]:
# Core libraries
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import time
import warnings
warnings.filterwarnings('ignore')

# Deep learning libraries
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

# IRST Library - All available models
from irst_library import IRSTDetector
from irst_library.datasets import SIRSTDataset
from irst_library.models import (
    SERANKNet, ACMNet, MSHNet, 
    UNet, UNetPlusPlus, DeepLabV3Plus,
    TransUNet, SwinUNet, SegFormer
)
from irst_library.evaluation import IRSTEvaluator
from irst_library.utils import (
    visualize_detection, plot_metrics, 
    calculate_model_complexity, profile_model
)

# Utilities
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm.auto import tqdm

# Set up environment
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🚀 Model Zoo Setup Complete!")
print(f"   💻 Device: {device}")
print(f"   🧠 PyTorch Version: {torch.__version__}")

# Model Zoo Configuration
MODEL_ZOO = {
    'serank': {
        'class': SERANKNet,
        'name': 'SERANKNet',
        'description': 'Search and ranking network with attention',
        'category': 'flagship',
        'params': {'in_channels': 1, 'num_classes': 1, 'use_attention': True}
    },
    'acm': {
        'class': ACMNet,
        'name': 'ACMNet', 
        'description': 'Asymmetric contextual modulation network',
        'category': 'flagship',
        'params': {'in_channels': 1, 'num_classes': 1}
    },
    'msh': {
        'class': MSHNet,
        'name': 'MSHNet',
        'description': 'Multi-scale hybrid network',
        'category': 'hybrid',
        'params': {'in_channels': 1, 'num_classes': 1}
    },
    'unet': {
        'class': UNet,
        'name': 'U-Net',
        'description': 'Classic encoder-decoder architecture',
        'category': 'classic',
        'params': {'in_channels': 1, 'n_classes': 1}
    },
    'unet++': {
        'class': UNetPlusPlus,
        'name': 'U-Net++',
        'description': 'Nested U-Net with skip connections',
        'category': 'classic',
        'params': {'in_channels': 1, 'n_classes': 1}
    },
    'deeplabv3+': {
        'class': DeepLabV3Plus,
        'name': 'DeepLabV3+',
        'description': 'Atrous spatial pyramid pooling',
        'category': 'backbone',
        'params': {'in_channels': 1, 'num_classes': 1}
    },
    'transunet': {
        'class': TransUNet,
        'name': 'TransUNet',
        'description': 'Transformer-based U-Net',
        'category': 'transformer',
        'params': {'in_channels': 1, 'num_classes': 1}
    },
    'swin_unet': {
        'class': SwinUNet,
        'name': 'Swin-UNet',
        'description': 'Swin transformer for segmentation',
        'category': 'transformer',
        'params': {'in_channels': 1, 'num_classes': 1}
    }
}

print(f"📚 Available Models: {len(MODEL_ZOO)}")
for key, info in MODEL_ZOO.items():
    print(f"   🎯 {info['name']} ({info['category']}): {info['description']}")

# Load test dataset for evaluation
test_transform = A.Compose([
    A.Normalize(mean=[0.485], std=[0.229], max_pixel_value=255.0),
    ToTensorV2()
], additional_targets={'mask': 'mask'})

test_dataset = SIRSTDataset(
    root="./data/SIRST",
    split="test", 
    transform=test_transform
)

test_loader = DataLoader(
    test_dataset,
    batch_size=8,
    shuffle=False,
    num_workers=4
)

print(f"📊 Test dataset loaded: {len(test_dataset)} samples")